# Les 10: Maximum Likelihood en Cross-Entropy

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 10.0 Recap en Motivatie

We hebben nu alle bouwstenen:
- **Lineaire algebra**: hoe data door een netwerk stroomt
- **Calculus**: hoe we gradiënten berekenen en optimaliseren
- **Kansrekening**: hoe we outputs interpreteren als kansen

Nu verbinden we alles. De centrale vraag is: **waarom gebruiken we cross-entropy loss?**

Het antwoord komt uit de statistiek: **Maximum Likelihood Estimation (MLE)**. We trainen een neuraal netwerk om de parameters te vinden die de waargenomen data het meest waarschijnlijk maken.

En het blijkt dat het maximaliseren van de likelihood wiskundig equivalent is aan het minimaliseren van de cross-entropy loss!

## 10.1 Leerdoelen

Na deze les begrijp je het principe van Maximum Likelihood Estimation. Je begrijpt de link tussen likelihood en cross-entropy. Je kunt uitleggen waarom we cross-entropy gebruiken voor classificatie. Je begrijpt de link tussen MSE en Gaussische likelihood. Je kunt de theoretische basis van loss functions uitleggen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize_scalar

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

## 10.2 Maximum Likelihood Estimation

### Het idee

Gegeven data D en een model met parameters θ, zoek de parameters die de data het meest waarschijnlijk maken:

θ* = argmax_θ P(D | θ)

De functie P(D | θ) noemen we de **likelihood**.

### Voorbeeld: Schat de kans op "kop"

Stel je gooit een munt 10 keer en krijgt 7 keer kop. Wat is de meest waarschijnlijke waarde van p (kans op kop)?

In [ ]:
# Data: 7 keer kop, 3 keer munt uit 10 worpen
n = 10
k = 7  # Aantal keer kop

# Likelihood functie: P(k kop | p) = C(n,k) * p^k * (1-p)^(n-k)
def likelihood(p, k, n):
    return stats.binom.pmf(k, n, p)

# Plot de likelihood voor verschillende waarden van p
p_values = np.linspace(0.01, 0.99, 100)
likelihoods = [likelihood(p, k, n) for p in p_values]

# Vind het maximum
p_mle = k / n  # Analytische oplossing voor binomiaal

plt.figure(figsize=(10, 5))
plt.plot(p_values, likelihoods, 'b-', linewidth=2)
plt.axvline(x=p_mle, color='r', linestyle='--', label=f'MLE: p = {p_mle}')
plt.xlabel('p (kans op kop)', fontsize=12)
plt.ylabel('Likelihood P(data | p)', fontsize=12)
plt.title(f'Likelihood functie voor {k} kop uit {n} worpen', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Maximum Likelihood Estimate: p = {p_mle}")
print(f"Dit is simpelweg k/n = {k}/{n}!")

### Log-Likelihood

In de praktijk werken we met de **log-likelihood** omdat:
1. Producten worden sommen (makkelijker te berekenen)
2. Numeriek stabieler (geen underflow)
3. Log is monotoon, dus maximum blijft hetzelfde

log P(D | θ) = Σᵢ log P(xᵢ | θ)

In [ ]:
# Log-likelihood
def log_likelihood(p, k, n):
    if p <= 0 or p >= 1:
        return -np.inf
    return k * np.log(p) + (n - k) * np.log(1 - p)

log_likelihoods = [log_likelihood(p, k, n) for p in p_values]

plt.figure(figsize=(10, 5))
plt.plot(p_values, log_likelihoods, 'b-', linewidth=2)
plt.axvline(x=p_mle, color='r', linestyle='--', label=f'MLE: p = {p_mle}')
plt.xlabel('p (kans op kop)', fontsize=12)
plt.ylabel('Log-Likelihood', fontsize=12)
plt.title('Log-Likelihood functie', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## 10.3 MLE voor een Normaal Verdeeld Model

Stel we hebben data {x₁, x₂, ..., xₙ} die we modelleren als N(μ, σ²). Wat zijn de MLE's van μ en σ²?

De log-likelihood is:

log L = -n/2 · log(2πσ²) - 1/(2σ²) · Σᵢ(xᵢ - μ)²

Door de afgeleiden naar μ en σ² gelijk aan nul te stellen, krijgen we:
- μ_MLE = (1/n) · Σᵢ xᵢ = steekproefgemiddelde
- σ²_MLE = (1/n) · Σᵢ(xᵢ - μ)² = steekproefvariantie

In [ ]:
# Genereer data uit N(5, 2²)
true_mu = 5
true_sigma = 2
n_samples = 100

data = np.random.normal(true_mu, true_sigma, n_samples)

# MLE schattingen
mu_mle = np.mean(data)
sigma2_mle = np.var(data)  # N in de noemer (MLE), niet N-1
sigma_mle = np.sqrt(sigma2_mle)

print(f"Werkelijke waarden: μ = {true_mu}, σ = {true_sigma}")
print(f"MLE schattingen: μ = {mu_mle:.4f}, σ = {sigma_mle:.4f}")

# Visualisatie
x = np.linspace(data.min() - 2, data.max() + 2, 100)
pdf_true = stats.norm.pdf(x, true_mu, true_sigma)
pdf_mle = stats.norm.pdf(x, mu_mle, sigma_mle)

plt.figure(figsize=(10, 5))
plt.hist(data, bins=20, density=True, alpha=0.5, label='Data')
plt.plot(x, pdf_true, 'g-', linewidth=2, label=f'Werkelijk: N({true_mu}, {true_sigma}²)')
plt.plot(x, pdf_mle, 'r--', linewidth=2, label=f'MLE: N({mu_mle:.2f}, {sigma_mle:.2f}²)')
plt.xlabel('x')
plt.ylabel('Dichtheid')
plt.title('MLE voor Normale Verdeling')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 10.4 Van MLE naar Cross-Entropy Loss

### Classificatie als MLE

In classificatie modelleert ons netwerk P(y | x; θ), de kans op klasse y gegeven input x en parameters θ.

Voor een dataset {(x₁, y₁), ..., (xₙ, yₙ)} is de likelihood:

L(θ) = ∏ᵢ P(yᵢ | xᵢ; θ)

En de log-likelihood:

log L(θ) = Σᵢ log P(yᵢ | xᵢ; θ)

### Negative Log-Likelihood = Cross-Entropy

Maximaliseren van log-likelihood is equivalent aan minimaliseren van de **negative log-likelihood (NLL)**:

NLL = -Σᵢ log P(yᵢ | xᵢ; θ)

Dit is precies de **cross-entropy loss**!

In [ ]:
# Demonstratie: NLL = Cross-Entropy

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=-1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=-1, keepdims=True)

# Stel: 4-klasse classificatie, 5 samples
n_samples = 5
n_classes = 4

# True labels (one-hot)
y_true = np.array([0, 2, 1, 3, 0])

# Model voorspellingen (softmax outputs)
logits = np.random.randn(n_samples, n_classes)
y_pred = softmax(logits)

print("Softmax outputs (P(klasse | x)):")
for i in range(n_samples):
    print(f"  Sample {i}: {y_pred[i]} → True label: {y_true[i]}")
print()

# Negative Log-Likelihood
nll = 0
for i in range(n_samples):
    prob_correct = y_pred[i, y_true[i]]
    nll -= np.log(prob_correct + 1e-10)
    print(f"  Sample {i}: P(y={y_true[i]}) = {prob_correct:.4f}, -log(P) = {-np.log(prob_correct):.4f}")

print(f"\nNegative Log-Likelihood = {nll:.4f}")
print(f"Cross-Entropy Loss (gemiddeld) = {nll/n_samples:.4f}")

## 10.5 Cross-Entropy Loss in Detail

### Definitie

De cross-entropy tussen de ware verdeling p en de voorspelde verdeling q is:

H(p, q) = -Σᵢ pᵢ · log(qᵢ)

### Voor classificatie

De ware verdeling p is een one-hot vector (1 bij de juiste klasse, 0 elders). Dus:

H(p, q) = -log(q_juiste_klasse)

Dit is precies wat we hierboven berekenden!

In [ ]:
# Cross-Entropy Loss implementatie

def cross_entropy_loss(y_pred, y_true):
    """
    Bereken cross-entropy loss.
    y_pred: (n_samples, n_classes) - softmax outputs
    y_true: (n_samples,) - class indices
    """
    n_samples = len(y_true)
    # Pak de voorspelde kans voor de juiste klasse
    probs_correct = y_pred[np.arange(n_samples), y_true]
    # Negative log
    loss = -np.mean(np.log(probs_correct + 1e-10))
    return loss

# Test
loss = cross_entropy_loss(y_pred, y_true)
print(f"Cross-Entropy Loss: {loss:.4f}")

In [ ]:
# Visualiseer: hoe verandert de loss met de voorspelde kans?

p_correct = np.linspace(0.01, 0.99, 100)
loss_values = -np.log(p_correct)

plt.figure(figsize=(10, 5))
plt.plot(p_correct, loss_values, 'b-', linewidth=2)
plt.xlabel('P(juiste klasse)', fontsize=12)
plt.ylabel('Loss = -log(P)', fontsize=12)
plt.title('Cross-Entropy Loss als functie van voorspelde kans', fontsize=14)
plt.grid(True, alpha=0.3)

# Markeer enkele punten
for p in [0.1, 0.5, 0.9]:
    plt.scatter([p], [-np.log(p)], s=100, zorder=5)
    plt.annotate(f'P={p}, Loss={-np.log(p):.2f}', (p, -np.log(p)), 
                 textcoords='offset points', xytext=(10,10), fontsize=10)

plt.show()

print("Observaties:")
print("  - Loss → 0 als P(juist) → 1 (perfecte voorspelling)")
print("  - Loss → ∞ als P(juist) → 0 (foute voorspelling)")
print("  - De loss is asymmetrisch: foute voorspellingen worden zwaar bestraft")

## 10.6 Binaire Cross-Entropy

Voor binaire classificatie (twee klassen) vereenvoudigt cross-entropy tot:

BCE = -[y · log(p) + (1-y) · log(1-p)]

waarbij:
- y ∈ {0, 1} is het echte label
- p ∈ [0, 1] is de voorspelde kans op klasse 1

In [ ]:
# Binary Cross-Entropy visualisatie

def binary_cross_entropy(y_true, y_pred):
    return -(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10))

p = np.linspace(0.01, 0.99, 100)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# y = 1 (positieve klasse)
loss_y1 = binary_cross_entropy(1, p)
axes[0].plot(p, loss_y1, 'b-', linewidth=2)
axes[0].set_xlabel('Voorspelde P(y=1)', fontsize=12)
axes[0].set_ylabel('BCE Loss', fontsize=12)
axes[0].set_title('True label y = 1', fontsize=14)
axes[0].grid(True, alpha=0.3)

# y = 0 (negatieve klasse)
loss_y0 = binary_cross_entropy(0, p)
axes[1].plot(p, loss_y0, 'r-', linewidth=2)
axes[1].set_xlabel('Voorspelde P(y=1)', fontsize=12)
axes[1].set_ylabel('BCE Loss', fontsize=12)
axes[1].set_title('True label y = 0', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Interpretatie:")
print("  Links (y=1): Loss laag als we hoge P(y=1) voorspellen")
print("  Rechts (y=0): Loss laag als we lage P(y=1) voorspellen")

## 10.7 MSE Loss en Gaussische Likelihood

Voor regressie gebruiken we vaak Mean Squared Error (MSE). Dit komt ook uit MLE!

Als we aannemen dat de data normaal verdeeld is rond de voorspelling:

y ~ N(f(x; θ), σ²)

Dan is de negative log-likelihood:

NLL = (1/2σ²) · Σᵢ(yᵢ - f(xᵢ; θ))² + constante

Dit is proportioneel aan de **MSE loss**!

In [ ]:
# Demonstratie: MSE = NLL onder Gaussische aanname

# Genereer regressie data
np.random.seed(42)
n_points = 50
X = np.linspace(0, 10, n_points)
y_true = 2 * X + 3  # Echte relatie
noise_std = 2
y = y_true + np.random.normal(0, noise_std, n_points)

# Simpel lineair model: y_pred = w*x + b
# Vind MLE (= minimaliseer MSE)

def mse_loss(params, X, y):
    w, b = params
    y_pred = w * X + b
    return np.mean((y - y_pred)**2)

def gaussian_nll(params, X, y, sigma=1):
    w, b = params
    y_pred = w * X + b
    n = len(y)
    nll = (1/(2*sigma**2)) * np.sum((y - y_pred)**2) + (n/2) * np.log(2*np.pi*sigma**2)
    return nll

# Grid search voor w en b
w_range = np.linspace(0, 4, 50)
b_range = np.linspace(0, 6, 50)
W, B = np.meshgrid(w_range, b_range)

MSE = np.zeros_like(W)
NLL = np.zeros_like(W)

for i in range(len(b_range)):
    for j in range(len(w_range)):
        MSE[i,j] = mse_loss([W[i,j], B[i,j]], X, y)
        NLL[i,j] = gaussian_nll([W[i,j], B[i,j]], X, y, sigma=noise_std)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

c1 = axes[0].contour(W, B, MSE, levels=20)
axes[0].clabel(c1, inline=True, fontsize=8)
axes[0].scatter([2], [3], color='red', s=100, marker='*', label='Optimum', zorder=5)
axes[0].set_xlabel('w')
axes[0].set_ylabel('b')
axes[0].set_title('MSE Loss')
axes[0].legend()

c2 = axes[1].contour(W, B, NLL, levels=20)
axes[1].clabel(c2, inline=True, fontsize=8)
axes[1].scatter([2], [3], color='red', s=100, marker='*', label='Optimum', zorder=5)
axes[1].set_xlabel('w')
axes[1].set_ylabel('b')
axes[1].set_title('Gaussian NLL')
axes[1].legend()

plt.tight_layout()
plt.show()

print("MSE en Gaussian NLL hebben dezelfde optimale parameters!")
print("Ze zijn proportioneel aan elkaar (verschillen alleen in schaal en constante).")

## 10.8 Vergelijking: MSE vs Cross-Entropy

| Eigenschap | MSE Loss | Cross-Entropy Loss |
|------------|----------|--------------------|
| Aanname | Gaussische ruis | Categorische verdeling |
| Gebruik | Regressie | Classificatie |
| Output | Continu | Kansen (softmax) |
| Gradiënt | Proportioneel aan fout | Sterker voor foute voorspellingen |
| MLE basis | N(y_pred, σ²) | Categorisch(softmax(z)) |

In [ ]:
# Vergelijk gradiënten

# Voor classificatie: vergelijk MSE vs CE gradiënt
# Bij MSE: ∂L/∂z ∝ (y_pred - y_true)
# Bij CE: ∂L/∂z = y_pred - y_true (voor softmax + CE)

# Stel: true label is klasse 0, model voorspelt [p, 1-p]
p_values = np.linspace(0.01, 0.99, 100)

# MSE gradient (vereenvoudigd)
mse_grad = 2 * (p_values - 1)  # Als we p willen verhogen naar 1

# CE gradient
ce_grad = p_values - 1  # softmax + CE geeft dit elegante resultaat

# -log(p) derivative
nll_grad = -1 / p_values  # Gradiënt van -log(p) naar p

plt.figure(figsize=(10, 5))
plt.plot(p_values, np.abs(mse_grad), 'b-', linewidth=2, label='|MSE gradient|')
plt.plot(p_values, np.abs(ce_grad), 'r-', linewidth=2, label='|CE gradient| (softmax)')
plt.xlabel('Voorspelde kans voor juiste klasse', fontsize=12)
plt.ylabel('|Gradiënt|', fontsize=12)
plt.title('Gradiënt sterkte: MSE vs Cross-Entropy', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("Observatie:")
print("  CE met softmax geeft een constante gradiënt ongeacht hoe zeker het model is.")
print("  Dit zorgt voor stabielere training bij classificatie.")

## 10.9 Praktische Implementatie

Laten we een complete classificatie pipeline implementeren met cross-entropy loss.

In [ ]:
# Complete classificatie met CE loss

class LogisticRegression:
    """Logistische regressie met gradient descent."""
    
    def __init__(self, n_features, n_classes):
        self.W = np.random.randn(n_features, n_classes) * 0.01
        self.b = np.zeros(n_classes)
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def forward(self, X):
        self.z = X @ self.W + self.b
        self.probs = self.softmax(self.z)
        return self.probs
    
    def compute_loss(self, y_true):
        n = len(y_true)
        log_probs = np.log(self.probs[np.arange(n), y_true] + 1e-10)
        return -np.mean(log_probs)
    
    def backward(self, X, y_true):
        n = len(y_true)
        # Gradiënt van softmax + CE: y_pred - y_true
        dz = self.probs.copy()
        dz[np.arange(n), y_true] -= 1
        dz /= n
        
        self.dW = X.T @ dz
        self.db = np.sum(dz, axis=0)
    
    def update(self, lr):
        self.W -= lr * self.dW
        self.b -= lr * self.db
    
    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)
    
    def accuracy(self, X, y):
        return np.mean(self.predict(X) == y)

In [ ]:
# Test op MNIST (subset)
from sklearn.datasets import load_digits

digits = load_digits()
X, y = digits.data, digits.target

# Normaliseer
X = X / 16.0

# Split
n_train = 1200
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

print(f"Training: {X_train.shape}, Test: {X_test.shape}")

# Train
model = LogisticRegression(n_features=64, n_classes=10)

lr = 1.0
n_epochs = 100
losses = []
accs = []

for epoch in range(n_epochs):
    model.forward(X_train)
    loss = model.compute_loss(y_train)
    losses.append(loss)
    
    model.backward(X_train, y_train)
    model.update(lr)
    
    acc = model.accuracy(X_test, y_test)
    accs.append(acc)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Test Acc = {acc:.4f}")

print(f"\nFinale test accuracy: {accs[-1]*100:.2f}%")

In [ ]:
# Visualiseer training
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(losses, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Cross-Entropy Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(accs, 'r-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Accuracy')
axes[1].set_title('Test Accuracy')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10.10 Samenvatting

### Kernconcepten

**Maximum Likelihood Estimation (MLE)** vindt de parameters die de waargenomen data het meest waarschijnlijk maken. We maximaliseren P(data | parameters).

**Log-likelihood** wordt gebruikt om producten te vervangen door sommen. Maximaliseren van log-likelihood = minimaliseren van negative log-likelihood.

**Cross-Entropy Loss = Negative Log-Likelihood** voor classificatie. Het volgt direct uit het MLE principe toegepast op categorische verdelingen.

**MSE Loss = Gaussian NLL** voor regressie. Het volgt uit MLE onder de aanname van normaal verdeelde ruis.

### De grote lijn

We hebben nu het complete plaatje:

1. **Forward pass** (lineaire algebra): data → logits → softmax → kansen
2. **Loss** (kansrekening): cross-entropy meet hoe goed de voorspelde verdeling is
3. **Backward pass** (calculus): bereken gradiënten via backprop
4. **Update** (optimalisatie): gradient descent past parameters aan

### Einde Deel 3

Met deze les sluiten we de theoretische basis af. We hebben de wiskundige fundamenten van neurale netwerken volledig behandeld. In Deel 4 passen we alles toe in praktische projecten!

---

**Mathematical Foundations** | Les 10 van 12 | IT & Artificial Intelligence

---